In [ ]:
from neuroscience_news.NeuroscienceNewsScraper import neuro_genres
from neuroscience_news.NeuroscienceNewsScraper import NeuroscienceNewsArticleScraper
from neuroscience_news.NeuroscienceNewsScraper import NeuroscienceNewsSiteScraper

In [ ]:
from tqdm import tqdm_notebook as tqdm
from joblib import Parallel, delayed
from datetime import datetime

# Scrape all of the genres for article urls

In [ ]:
site_scraper = NeuroscienceNewsSiteScraper()

In [ ]:
neuro_urls = []
for key in tqdm(neuro_genres):
    genre_url = neuro_genres[key][0]
    genre_page_count = neuro_genres[key][1]
    
    neuro_urls.extend(site_scraper.scrape_genre_for_article_urls(genre_url, genre_page_count))

### Save list of urls to txt file

In [ ]:
with open('./data/urls/neuroscience_news_urls.txt', 'w') as text_file:
    for url in neuro_urls:
        text_file.write("{}\n".format(url))

# Scrape Articles

In [ ]:
article_scraper = NeuroscienceNewsArticleScraper()

### With urls, get raw html of article

In [ ]:
neuro_htmls = Parallel(n_jobs=-1)(delayed(article_scraper.get_article_html)(url) for url in tqdm(neuro_urls))

### Save HTML to file for later use

In [ ]:
def save_html(html):
    currentDT = datetime.now()
    time = currentDT.strftime("%H%M%S%f")
    
    article_name = "article_{}".format(time)
    
    f = open("./data/html/{}.html".format(article_name), "wb")
    f.write(html)
    f.close()

Parallel(n_jobs=-1)(delayed(save_html)(html) for html in tqdm(neuro_htmls))

### Scrape Individual Article Information

In [ ]:
neuro_articles = Parallel(n_jobs=-1)(delayed(article_scraper.scrape_article)
                                         (html) for html in tqdm(neuro_htmls))

In [ ]:
def clean_list(article):
    if article:
        return True
    else:
        return False

In [ ]:
clean_neuro_articles = list(filter(clean_list, neuro_articles))

In [ ]:
len(clean_neuro_articles)

# Store Articles in Mongo

In [ ]:
from mongoengine import connect

In [ ]:
from mongo_processor.NeuroscienceNewsMongoConnector import NeuroscienceNewsMongoConnector

In [ ]:
#http://docs.mongoengine.org/guide/connecting.html
collection_name = "Neuroscience_News"
connect(collection_name)

In [ ]:
article_saver = NeuroscienceNewsMongoConnector(collection_name)
article_saver.save_neuroscience_articles(clean_neuro_articles)

In [ ]:
# http://docs.mongoengine.org/guide/defining-documents.html
NeuroscienceNewsArticle.objects.count()